# Dots and Boxes in Python
Dots and Boxes is a simple Strategy game developed in the 1880s in France.
This program will attempt to recreate it in Python and apply the Decision Tree algorithm to the results, with a limited ply-depth to ensure that the program does not blow up Google Colab.

## Dots and Boxes logic
The game dots and boxes is a game that involves conquering territory from the opposing player. When all four edges are placed around a box, the box is considered "Won" on behalf of the entity that took the box.

A box is only considered "Won" when the final edge is filled in. The entity that filled in the final edge wins the box in question.

[Link to Numberphile Video containing "always win" logic.](https://youtu.be/KboGyIilP6k)

In [ ]:
import random

In [ ]:
class SearchTreeNode:
  def __init__(self, game_board, current_player, ply=0, max_depth=5):
    self.children = []
    self.value_is_assigned = False
    self.ply_depth = ply
    self.current_board = game_board
    self.move_for = current_player

    if self.ply_depth < max_depth and self.current_board.state_of_board() == "U":
      self.generate_children(max_depth)
    else: # Reassign values based on evaluation function.
      if self.current_board.state_of_board() == "D":
        self.value = 0
      else:
        if ((self.ply_depth % 2) == 0):
          self.value = -1
        else:
          self.value = 1
      self.value_is_assigned = True

  def min_max_value(self, max_depth):
    if self.value_is_assigned:
      return self.value
    self.children = sorted(self.children, key = lambda x:x.min_max_value(5))
    if ((self.ply_depth % 2) == 0):
      self.value = self.children[-1].value
    else:
      self.value = self.children[0].value
    self.value_is_assigned = True
    return self.value

  def generate_children(self, max_depth):
    for board_for_next_move in self.current_board.all_possible_moves():
      self.children.append(SearchTreeNode(board_for_next_move, self.current_board.change_player(), ply = self.ply_depth +1, max_depth = max_depth))

In [ ]:
'''Map of the Game board for reference. It's long for easier deciphering of indices.'''
#      [0,1]     [0,3]     [0,5]
# [1,0]     [1,2]     [1,4]     [1,6]
#      [2,1]     [2,3]     [2,5]
# [3,0]     [3,2]     [3,4]     [3,6]
#      [4,1]     [4,3]     [4,5]
class DotsAndBoxes:
  def __init__(self, newBoard = None): # Implement Game State for algorithm.
    self.current_player = 1
    self.player_score = 0
    self.computer_score = 0
    #self.state = self.state_of_board() # this is causing a crash, for some reason.
    if newBoard is None:
      self.game_board = [
        ["*", " ", "*", " ", "*", " ", "*"],
        [" ", "0", " ", "0", " ", "0", " "],
        ["*", " ", "*", " ", "*", " ", "*"],
        [" ", "0", " ", "0", " ", "0", " "],
        ["*", " ", "*", " ", "*", " ", "*"]
      ]
    else: # Otherwise initialize this game board instance.
      self.game_board = newBoard

  def display_board(self):
    for row in self.game_board:
      print(''.join(row))

  def state_of_board(self): # Logic here works for Tic Tac Toe, not Dots and Boxes
    if self.return_available_moves():
      return "U"
    else:
      return "D"

  def return_available_moves(self):
    remaining_moves = []
    for row_index in range(len(self.game_board)):
      for col_index in range(len(self.game_board[row_index])):
        if self.game_board[row_index][col_index] == " ":
          remaining_moves.append((row_index, col_index))
    return remaining_moves

  def all_possible_moves(self):
    remaining_moves = self.return_available_moves()
    all_moves = []
    for move in remaining_moves:
      new_board = [row[:] for row in self.game_board]
      new_board[move[0]][move[1]] = "-"
      all_moves.append(DotsAndBoxes(new_board))
    return all_moves

  def check_if_box_completed(self, row, col): # Checks if placing a line completes more than one box. Checks run after every placed line.
    boxes_completed = 0
    possible_boxes = [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]
    for r, c in possible_boxes: # Checks whether column and row are within the game board. Explanation below.
      if 0 <= r < len(self.game_board) and 0 <= c < len(self.game_board[r]): # Necessary because of zero values, E.G (0,1) will evaluate to (-0,1) for possible_boxes[(row - 1, col)]
        if self.is_box_complete(r, c):
          self.game_board[r][c] = str(self.current_player)
          boxes_completed += 1
    return boxes_completed

  def is_box_complete(self, row, col): # Checks north south east west values. Returns True if all occupied.
    if self.game_board[row][col] != "0":
      return False
    if self.game_board[row-1][col] != "-":
      return False
    if self.game_board[row+1][col] != "-":
      return False
    if self.game_board[row][col-1] != "-":
      return False
    if self.game_board[row][col+1] != "-":
      return False
    return True

  def count_score(self):
    self.player_score = sum(row.count("1") for row in self.game_board)
    self.computer_score = sum(row.count("2") for row in self.game_board)
    return self.player_score, self.computer_score

  def change_player(self):
    self.current_player = 3 - self.current_player

  def player_move(self):
    remaining_moves = self.return_available_moves()
    player_score, computer_score = self.count_score()
    print(f"Player Score: {player_score}")
    print(f"Computer Score: {computer_score}")
    print(f"Remaining Moves: {remaining_moves}")
    row = int(input("Please enter a row for the move (0-4): "))
    col = int(input("Please enter a column for the move (0-4): "))
    if (row, col) not in remaining_moves:
      print("Invalid move. Please try again.")
      return False
    self.add_line(row, col)
    boxes_completed = self.check_if_box_completed(row, col)
    if boxes_completed == 0:
      self.change_player()
    return True

  def add_line(self, row, col):
    self.game_board[row][col] = "-"

  def computer_move(self):
    search_tree = SearchTreeNode(self, self.current_player)
    search_tree.min_max_value(5)
    if self.current_player == 2:
      best_child = max(search_tree.children, key=lambda x: x.value)
    else:
      best_child = min(search_tree.children, key=lambda x: x.value)
    cb = best_child.current_board # Get best move
    for r in range(len(self.game_board)):
      for c in range(len(self.game_board[r])):
        if self.game_board[r][c] != cb.game_board[r][c]:
          move = r, c
    self.add_line(move[0], move[1]) # Add move
    boxes_completed = self.check_if_box_completed(move[0], move[1])
    if boxes_completed == 0:
      self.change_player()

  def play_game(self):
      self.display_board()
      while self.return_available_moves():
        if self.current_player == 1:
          valid_move = self.player_move()
          while not valid_move:
            valid_move = self.player_move()
        else:
          self.computer_move()
          print("Computer made a move.")
        self.display_board()
      player_score, computer_score = self.count_score()
      if player_score > computer_score:
        print("Congratulations! You won!")
      elif player_score < computer_score:
        print("Computer wins!")
      else:
        print("It's a draw!")

In [ ]:
cb = DotsAndBoxes()

In [ ]:
game = DotsAndBoxes()
for g in game.all_possible_moves():
  g.display_board()

*-* * *
 0 0 0 
* * * *
 0 0 0 
* * * *
* *-* *
 0 0 0 
* * * *
 0 0 0 
* * * *
* * *-*
 0 0 0 
* * * *
 0 0 0 
* * * *
* * * *
-0 0 0 
* * * *
 0 0 0 
* * * *
* * * *
 0-0 0 
* * * *
 0 0 0 
* * * *
* * * *
 0 0-0 
* * * *
 0 0 0 
* * * *
* * * *
 0 0 0-
* * * *
 0 0 0 
* * * *
* * * *
 0 0 0 
*-* * *
 0 0 0 
* * * *
* * * *
 0 0 0 
* *-* *
 0 0 0 
* * * *
* * * *
 0 0 0 
* * *-*
 0 0 0 
* * * *
* * * *
 0 0 0 
* * * *
-0 0 0 
* * * *
* * * *
 0 0 0 
* * * *
 0-0 0 
* * * *
* * * *
 0 0 0 
* * * *
 0 0-0 
* * * *
* * * *
 0 0 0 
* * * *
 0 0 0-
* * * *
* * * *
 0 0 0 
* * * *
 0 0 0 
*-* * *
* * * *
 0 0 0 
* * * *
 0 0 0 
* *-* *
* * * *
 0 0 0 
* * * *
 0 0 0 
* * *-*


In [ ]:
cb.display_board()

* * * *
 0 0 0 
* * * *
 0 0 0 
* * * *


In [ ]:
st = SearchTreeNode(cb,1)

In [ ]:
game = DotsAndBoxes()
game.play_game()

* * * *
 0 0 0 
* * * *
 0 0 0 
* * * *
Player Score: 0
Computer Score: 0
Remaining Moves: [(0, 1), (0, 3), (0, 5), (1, 0), (1, 2), (1, 4), (1, 6), (2, 1), (2, 3), (2, 5), (3, 0), (3, 2), (3, 4), (3, 6), (4, 1), (4, 3), (4, 5)]
Please enter a row for the move (0-4): 0
Please enter a column for the move (0-4): 1
*-* * *
 0 0 0 
* * * *
 0 0 0 
* * * *
Computer made a move.
*-*-* *
 0 0 0 
* * * *
 0 0 0 
* * * *
Player Score: 0
Computer Score: 0
Remaining Moves: [(0, 5), (1, 0), (1, 2), (1, 4), (1, 6), (2, 1), (2, 3), (2, 5), (3, 0), (3, 2), (3, 4), (3, 6), (4, 1), (4, 3), (4, 5)]
Please enter a row for the move (0-4): 3
Please enter a column for the move (0-4): 6
*-*-* *
 0 0 0 
* * * *
 0 0 0-
* * * *
Computer made a move.
*-*-*-*
 0 0 0 
* * * *
 0 0 0-
* * * *
Player Score: 0
Computer Score: 0
Remaining Moves: [(1, 0), (1, 2), (1, 4), (1, 6), (2, 1), (2, 3), (2, 5), (3, 0), (3, 2), (3, 4), (4, 1), (4, 3), (4, 5)]
Please enter a row for the move (0-4): 1
Please enter a column for the